<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=72a3f855a7ae181624a00eff1beda7182005bb08980287e4498ae1ff09c50bc3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 14:40:58
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 71.80 K (0.51%)
Current PnL: -20.54 L (-13.64%)
CY Booked + Current PnL: -9.10 L (-6.04%)
-------------------
Total profit:  1.42 L
Total loss:  -21.96 L
-------------------
Total Booked + Current PnL: 17.88 L (14.45%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.86 L (62.29%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.01,117342.0,16838.0,12274.0,3.22,16.75,10.46,28.96,79.0,1.37,0.84,56.41,MH,ATH,METALS
77,TTKPRESTIG,770.00,95.30,42.0,M-SC,1.69,84174.0,-16603.0,16700.0,0.86,-16.47,19.84,0.09,245.0,-0.99,0.60,10.59,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.97,189340.0,24670.0,18669.0,0.83,14.98,9.86,26.31,89.0,1.32,1.35,24.16,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
84,VOLTAS,1530.00,-2.76,45.0,H-MC,2.21,203910.0,12168.0,25591.0,0.44,6.35,12.55,19.69,99.0,0.48,1.45,13.57,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-17.45,58.0,H-SC,18.57,220840.0,-49921.0,379867.0,4.53,-18.44,172.01,121.86,133.0,-0.13,1.57,72.76,AR,ATH,IT
62,SATIN,274.00,-25.10,64.0,H-SC,4.70,231579.0,-44705.0,187926.0,4.32,-16.18,81.15,51.83,126.0,-0.24,1.65,12.81,XY24,NTT,FINANCE
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.01,117342.0,16838.0,12274.0,3.22,16.75,10.46,28.96,79.0,1.37,0.84,56.41,MH,ATH,METALS
24,DEN,75.00,99.48,48.0,M-SC,1.94,55968.0,-28496.0,64033.0,2.88,-33.74,114.41,42.07,237.0,-0.45,0.40,18.86,AR,NTT,ENTERTAINMENT
32,HINDZINC,730.22,31.71,68.0,M-LC,4.49,214324.0,9248.0,103326.0,2.36,4.51,48.21,54.89,52.0,0.09,1.53,29.69,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-185.23,74.0,M-SC,9.47,148140.0,-1080.0,145266.0,-2.94,-0.72,98.06,96.62,208.0,-0.01,1.05,60.32,XY25,NTT,FINANCE
66,SIS,528.00,2064.77,39.0,H-SC,3.69,87363.0,-23669.0,47281.0,-1.86,-21.32,54.12,21.26,156.0,-0.50,0.62,18.04,OX40N,NTT,MISC
69,SURYODAY,240.00,68.86,75.0,H-SC,3.76,155917.0,-23154.0,88888.0,-1.75,-12.93,57.01,36.71,135.0,-0.26,1.11,54.70,XR,NTT,BANKS
20,COALINDIA,497.97,5.97,41.0,L-LC,4.22,141525.0,-8936.0,42726.0,-1.54,-5.94,30.19,22.46,167.0,-0.21,1.01,11.07,XY25,ATH,MINING
49,MASFIN,398.61,-20.05,41.0,H-SC,11.56,90825.0,-7155.0,28755.0,-1.34,-7.30,31.66,22.05,152.0,-0.25,0.65,32.02,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1236.52,68.0,M-SC,18.23,181705.0,562.0,44899.0,-0.74,0.31,24.71,25.10,235.0,0.01,1.29,45.62,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-185.23,74.0,M-SC,9.47,148140.0,-1080.0,145266.0,-2.94,-0.72,98.06,96.62,208.0,-0.01,1.05,60.32,XY25,NTT,FINANCE
35,ICICIPRULI,790.0,-21.11,49.0,H-MC,2.16,135826.0,38.0,42717.0,0.74,0.03,31.45,31.48,107.0,0.00,0.97,12.15,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.08,129897.0,-12582.0,32098.0,-0.25,-8.83,24.71,13.70,163.0,-0.39,0.92,53.31,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,60.0,H-SC,1.85,226485.0,-43182.0,79519.0,0.98,-16.01,35.11,13.47,138.0,-0.54,1.61,14.55,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,95.30,42.0,M-SC,1.69,84174.0,-16603.0,16700.0,0.86,-16.47,19.84,0.09,245.0,-0.99,0.60,10.59,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,40.0,H-SC,1.48,141450.0,-34453.0,76482.0,0.72,-19.59,54.07,23.89,149.0,-0.45,1.01,22.16,OX40N,NTT,CERAMICS
66,SIS,528.0,2064.77,39.0,H-SC,3.69,87363.0,-23669.0,47281.0,-1.86,-21.32,54.12,21.26,156.0,-0.50,0.62,18.04,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1236.52,68.0,M-SC,18.23,181705.0,562.0,44899.0,-0.74,0.31,24.71,25.10,235.0,0.01,1.29,45.62,XY24,NTT,HEALTHCARE
60,SAIL,228.00,48.36,61.0,M-MC,11.92,233865.0,8903.0,158748.0,0.66,3.96,67.88,74.53,192.0,0.06,1.66,37.49,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,68.0,M-LC,4.49,214324.0,9248.0,103326.0,2.36,4.51,48.21,54.89,52.0,0.09,1.53,29.69,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.97,189340.0,24670.0,18669.0,0.83,14.98,9.86,26.31,89.0,1.32,1.35,24.16,X40N,NTT,AC
84,VOLTAS,1530.00,-2.76,45.0,H-MC,2.21,203910.0,12168.0,25591.0,0.44,6.35,12.55,19.69,99.0,0.48,1.45,13.57,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.01,117342.0,16838.0,12274.0,3.22,16.75,10.46,28.96,79.0,1.37,0.84,56.41,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,68.0,H-MC,10.06,192819.0,13011.0,99398.0,0.59,7.24,51.55,62.52,88.0,0.13,1.37,43.79,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,68.0,M-LC,4.49,214324.0,9248.0,103326.0,2.36,4.51,48.21,54.89,52.0,0.09,1.53,29.69,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.83,35.0,H-SC,9.39,125981.0,2645.0,131121.0,-0.91,2.14,104.08,108.45,119.0,0.02,0.90,24.98,AR,ATH,MISC
86,WIPRO,420.00,-15.15,41.0,M-LC,5.84,149420.0,-1525.0,110974.0,-0.03,-1.01,74.27,72.51,53.0,-0.01,1.06,4.95,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282251.0,-33591.0,146262.0,-0.12,-10.64,51.82,35.67,5.0,-0.23,2.01,1.75,X40N,ATH,FMCG
74,TCS,4476.75,-31.58,32.0,H-LC,12.49,272920.0,-73036.0,147895.0,-0.37,-21.11,54.19,21.64,1.0,-0.49,1.94,0.52,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.31,217586.0,-24338.0,87600.0,-0.26,-10.06,40.26,26.15,8.0,-0.28,1.55,2.82,X40,ATH,IT
5,ASIANPAINT,3465.66,-17.77,34.0,H-LC,5.81,207495.0,-44273.0,97481.0,0.95,-17.58,46.98,21.13,27.0,-0.45,1.48,11.61,X40,ATH,PAINTS
44,JIOFIN,387.00,-4.80,39.0,H-LC,12.83,258688.0,-8677.0,74140.0,0.25,-3.25,28.66,24.48,48.0,-0.12,1.84,49.94,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-1.76,48.0,H-LC,1.75,158045.0,-28050.0,75530.0,1.26,-15.07,47.79,25.51,15.0,-0.37,1.13,16.47,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.94,43.0,H-LC,1.80,230850.0,-15301.0,94510.0,0.14,-6.22,40.94,32.18,16.0,-0.16,1.64,27.59,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.17,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
51,NESTLEIND,1377.0,-11.02,47.0,H-LC,3.45,273874.0,8448.0,48339.0,0.34,3.18,17.65,21.40,11.0,0.17,1.95,10.04,XY25,NTT,FMCG
57,RELIANCE,1533.0,-15.17,39.0,H-LC,3.54,212550.0,2184.0,26590.0,-0.45,1.04,12.51,13.68,37.0,0.08,1.51,17.73,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,32.0,H-LC,12.49,272920.0,-73036.0,147895.0,-0.37,-21.11,54.19,21.64,1.0,-0.49,1.94,0.52,X40,ATH,IT
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282251.0,-33591.0,146262.0,-0.12,-10.64,51.82,35.67,5.0,-0.23,2.01,1.75,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.31,217586.0,-24338.0,87600.0,-0.26,-10.06,40.26,26.15,8.0,-0.28,1.55,2.82,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
40,INFY,2275.00,-21.03,39.0,H-LC,8.24,307849.0,-5039.0,176736.0,-0.03,-1.61,57.41,54.87,3.0,-0.03,2.19,4.93,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.76,48.0,H-LC,1.75,158045.0,-28050.0,75530.0,1.26,-15.07,47.79,25.51,15.0,-0.37,1.13,16.47,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,45.0,H-LC,3.91,208144.0,3644.0,37778.0,0.42,1.78,18.15,20.25,10.0,0.10,1.48,10.67,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.77,34.0,H-LC,5.81,207495.0,-44273.0,97481.0,0.95,-17.58,46.98,21.13,27.0,-0.45,1.48,11.61,X40,ATH,PAINTS
57,RELIANCE,1533.00,-15.17,39.0,H-LC,3.54,212550.0,2184.0,26590.0,-0.45,1.04,12.51,13.68,37.0,0.08,1.51,17.73,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,32.0,H-LC,12.49,272920.0,-73036.0,147895.0,-0.37,-21.11,54.19,21.64,1.0,-0.49,1.94,0.52,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.24,307849.0,-5039.0,176736.0,-0.03,-1.61,57.41,54.87,3.0,-0.03,2.19,4.93,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282251.0,-33591.0,146262.0,-0.12,-10.64,51.82,35.67,5.0,-0.23,2.01,1.75,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,47.0,H-LC,7.65,249888.0,-11731.0,130941.0,0.13,-4.48,52.40,45.57,7.0,-0.09,1.78,6.33,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,37.53,80256.0,-33293.0,73346.0,0.92,-29.32,91.39,35.27,268.0,-0.45,0.57,96.43,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,41.0,H-SC,11.56,90825.0,-7155.0,28755.0,-1.34,-7.30,31.66,22.05,152.0,-0.25,0.65,32.02,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.01,117342.0,16838.0,12274.0,3.22,16.75,10.46,28.96,79.0,1.37,0.84,56.41,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,12.08,129897.0,-12582.0,32098.0,-0.25,-8.83,24.71,13.70,163.0,-0.39,0.92,53.31,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,74.0,M-SC,9.47,148140.0,-1080.0,145266.0,-2.94,-0.72,98.06,96.62,208.0,-0.01,1.05,60.32,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1236.52,68.0,M-SC,18.23,181705.0,562.0,44899.0,-0.74,0.31,24.71,25.10,235.0,0.01,1.29,45.62,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,68.86,75.0,H-SC,3.76,155917.0,-23154.0,88888.0,-1.75,-12.93,57.01,36.71,135.0,-0.26,1.11,54.70,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,74.0,M-SC,9.47,148140.0,-1080.0,145266.0,-2.94,-0.72,98.06,96.62,208.0,-0.01,1.05,60.32,XY25,NTT,FINANCE
71,TANLA,1963.11,-17.45,58.0,H-SC,18.57,220840.0,-49921.0,379867.0,4.53,-18.44,172.01,121.86,133.0,-0.13,1.57,72.76,AR,ATH,IT
7,ATULAUTO,844.00,3618.92,51.0,M-SC,7.88,167093.0,-30039.0,119873.0,0.25,-15.24,71.74,45.57,236.0,-0.25,1.19,18.46,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.94
1,25,43.14
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.30
LC    31.96
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.43
X40N     11.21
XR       11.08
XY25     10.35
AR        8.19
OX40N     7.78
MH        1.65
X200      1.64
X5K       1.53
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.11
H-LC    25.40
H-MC    20.45
M-SC    13.75
M-LC     5.55
M-MC     2.97
L-SC     1.44
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.79,-5.95,40.92
IT,12.49,-19.79,85.10
FINANCE,10.43,-15.11,67.40
MISC,6.92,-18.13,83.81
BANKS,6.39,-10.90,69.24
PAINTS,5.60,-17.22,34.70
ELECTRICAL,5.31,-10.41,49.14
INSURANCE,3.75,-5.18,44.50
AUTO,3.56,-14.73,69.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221042.0,22
XR,1354079.0,14
AR,1182234.0,9
X40,822289.0,10
X40N,630593.0,9
OX40N,553998.0,10
XY25,482232.0,7
SR,259067.0,2
X5K,103326.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3705002.0,29
M-SC,1499817.0,17
H-LC,1251984.0,15
H-MC,1232797.0,15
M-LC,369545.0,4
M-MC,364707.0,2
L-SC,259100.0,3
L-MC,60005.0,1
L-LC,42726.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1214588.0      6
M-SC       XY24       847242.0      7
H-SC       AR         809185.0      5
           XR         783010.0      7
H-MC       XY24       565120.0      4
H-LC       X40        533041.0      5
M-MC       XY24       364707.0      2
H-SC       X40N       313956.0      4
           SR         259067.0      2
           OX40N      255157.0      4
H-LC       X40N       231443.0      3
H-MC       X40        216184.0      4
H-LC       AR         206471.0      2
H-MC       XY25       181861.0      2
L-SC       XR         170738.0      2
M-SC       AR         166578.0      2
M-LC       XY24       155245.0      2
M-SC       XY25       145266.0      1
           OX40N      137713.0      4
           XR         129954.0      2
H-LC       XY25       112379.0      3
M-LC       XR         110974.0      1
           X5K        103326.0      1
H-MC       XR          99398.0      1
H-LC       X200        94510.0      1
L-SC       OX40N       88362.0      1
H-MC       X40N        85194.0      2
H-LC       XY24        74140.0      1
M-SC       X40         73064.0      1
H-MC       OX40N       72766.0      1
H-SC       MH          70039.0      1
L-MC       XR          60005.0      1
L-LC       XY25        42726.0      1
H-MC       MH          12274.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
